In [1]:
import pandas as pd
import numpy as np
import folium

xls_trips_file = pd.ExcelFile('divvyTrips-2018-Q2.xlsx')
bikeData = xls_trips_file.parse()
xls_stations_file = pd.ExcelFile('Divvy_Stations_2017_Q1Q2.xlsx')
stationData = xls_stations_file.parse()


bikeData['Age'] = 2018 - bikeData['birthyear']
averageSubAge = bikeData['Age'].mean()

numSubMen = len(bikeData[bikeData['gender'] == 'Male'])
numSubWomen = len(bikeData) - numSubMen

avgTripDuration = bikeData['tripduration'].mean()

bikeGrouped = bikeData.groupby(by='bikeid')

fromStationGrouped = bikeData.groupby(by='from_station_id')

chiMap = folium.Map([41.8781, -87.6298], zoom_start=11, tiles='Stamen Toner')


locationVectorData = pd.merge(bikeData, stationData[['id', 'latitude', 'longitude']], left_on='from_station_id', right_on='id', how='inner')
locationVectorData = locationVectorData.drop(columns = ['id']).rename(columns={'latitude' : 'from_latitude', 'longitude' : 'from_longitude'})
locationVectorData = locationVectorData.drop(columns=['start_time', 'end_time', 'tripduration', 'usertype', 'gender', 'birthyear', 'Age'])
locationVectorData = pd.merge(locationVectorData, stationData[['id', 'latitude', 'longitude']], left_on='to_station_id', right_on='id', how='inner')
locationVectorData = locationVectorData.rename(columns={'latitude' : 'to_latitude', 'longitude' : 'to_longitude'})
locationVectorData = locationVectorData.drop(columns=['id'])

In [5]:
chiMap

In [40]:
def drawAllAvgTripVectors(bikeDataFrame, stationIDList, foliumMap):
    for id in stationIDList:
        stationSpecificData = bikeDataFrame[bikeDataFrame['from_station_id'] == id]
        originVector = np.asarray(stationSpecificData.head(1)[['from_latitude', 'from_longitude']])
        avgTripVector = getAvgTrip(stationSpecificData[['to_latitude', 'to_longitude']], originVector)
        endPoint = np.add(originVector, avgTripVector)[0]
        origin = originVector[0]
        drawVector = folium.PolyLine([origin, endPoint], color='red',opacity=0.7)
        drawVector.add_to(foliumMap)
        drawVectorHead(origin, endPoint, 'red', foliumMap)
    return foliumMap

def sumVectorList(vectorList):
    sum = np.array([0, 0])
    for vector in vectorList:
        sum = np.add(sum, vector)
    return sum

def twoColumnsToNormalizedVectors(twoColDataFrame, originVector):
    return [np.subtract(np.asarray((row[1], row[2])), originVector) for row in twoColDataFrame.itertuples()]

def normalizedTripsAvg(vectorList):
    return sumVectorList(vectorList) / len(vectorList)

def getAvgTrip(latLongColumns, originVector):
    return normalizedTripsAvg(twoColumnsToNormalizedVectors(latLongColumns, originVector))

#how can I access the attributes of a folium polyline?
def drawVectorHead(latLongOrigin, latLongEndPoint, color, foliumMap):
    arrowRotation = getBearing(latLongOrigin, latLongEndPoint) - 90
    folium.RegularPolygonMarker(location=latLongEndPoint,
                            fill_color=color,
                            number_of_sides=3,
                            rotation=arrowRotation,
                            radius=10).add_to(foliumMap)
    
def getBearing(point1, point2):
    longDifference = np.radians(point2[1] - point1[1])
    p1Lat = np.radians(point1[0])
    p2Lat = np.radians(point2[0])
    
    x = np.sin(longDifference) * np.cos(p2Lat)
    y = np.cos(p1Lat) * np.sin(p2Lat) - (np.sin(p1Lat) * np.cos(p2Lat) * np.cos(longDifference))
        
    bearing = np.degrees(np.arctan2(x, y))
    if (bearing < 0):
        return bearing + 360
    return bearing

In [42]:
chiMap = drawAllAvgTripVectors(locationVectorData, [22, 42, 71], chiMap)
chiMap

In [12]:
stationIDSet = set(locationVectorData['from_station_id'].tolist())
chiMap

In [40]:
chiMap

In [9]:
# calculating average trips for most popular evening stations
# input dataFrame should be ALL trips that involve the top 20 departure stations

In [10]:
#here is an example of drawing a triangle on a folium map; need to do some trig to calculate the rotation (starts due East)
folium.RegularPolygonMarker(location=(lat, long),
                            fill_color='blue',
                            number_of_sides=3,
                            radius=10, rotation=???).add_to(someMap)

SyntaxError: invalid syntax (<ipython-input-10-bda84928280b>, line 5)